In [116]:
# !pip install --upgrade ta

In [117]:
from decimal import ROUND_HALF_UP, Decimal

import numpy as np
import pandas as pd
from __future__ import division
import ta

In [118]:
df = pd.read_csv("../Input/train_files/stock_prices.csv")
# val_df = pd.read_csv("../Input/supplemental_files/stock_prices.csv")

# stock_list = pd.read_csv("../Input/stock_list.csv")


In [119]:
# df = pd.concat([train_df, val_df]).reset_index(drop=True)

In [120]:
stock_list = pd.read_csv("../Input/stock_list.csv")

In [121]:
stock_list = stock_list[['SecuritiesCode', 'Section/Products', 'NewMarketSegment', '33SectorCode', '17SectorCode', 'NewIndexSeriesSizeCode', 'Universe0']]
stock_list.replace('-', np.nan, inplace=True)

In [122]:
df = df.merge(stock_list, on='SecuritiesCode', how='left')

In [123]:
df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Section/Products,NewMarketSegment,33SectorCode,17SectorCode,NewIndexSeriesSizeCode,Universe0
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,First Section (Domestic),Prime Market,50,1,7,True
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,First Section (Domestic),Prime Market,50,1,4,True
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,First Section (Domestic),Prime Market,50,1,4,True
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,First Section (Domestic),Standard Market,50,1,7,True
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,First Section (Domestic),Prime Market,50,1,6,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,First Section (Domestic),Prime Market,6100,14,7,True
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,First Section (Domestic),Prime Market,6050,13,7,True
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,First Section (Domestic),Standard Market,6100,14,7,True
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,First Section (Domestic),Standard Market,6100,14,7,True


In [124]:
# df.drop(columns=['RowId'], inplace=True)

In [125]:
def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == "int":
                if (c_min > np.iinfo(np.int8).min
                        and c_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (c_min > np.iinfo(np.int16).min
                      and c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f"Mem. usage decreased to {end_mem:5.2f} MB" + \
        f" ({reduction * 100:.1f} % reduction)"
    if verbose:
        print(msg)

    return df


In [126]:
def get_time_info(df, is_onehot=False):
    datetime_series = pd.to_datetime(df['Date']).dt

    def get_weekday(dt_series, is_onehot=False):
        if is_onehot:
            return pd.get_dummies(dt_series.day_name())
        return dt_series.dayofweek

    def get_week(dt_series):
        return dt_series.isocalendar().week

    def get_month(dt_series, is_onehot=False):
        if is_onehot:
            return pd.get_dummies(dt_series.month_name())
        return dt_series.month

    weekday = get_weekday(datetime_series, is_onehot)
    week = get_week(datetime_series)
    month = get_month(datetime_series, is_onehot)

    if is_onehot:
        return pd.concat([df, weekday, week, month], axis=1)
    df['weekday'] = weekday
    df['week'] = week
    df['month'] = month
    return df

In [127]:
df = get_time_info(df)

In [128]:
print(df.shape)
print(df.isnull().sum())

(2332531, 21)
RowId                           0
Date                            0
SecuritiesCode                  0
Open                         7608
High                         7608
Low                          7608
Close                        7608
Volume                          0
AdjustmentFactor                0
ExpectedDividend          2313666
SupervisionFlag                 0
Target                        238
Section/Products                0
NewMarketSegment                0
33SectorCode                    0
17SectorCode                    0
NewIndexSeriesSizeCode     303034
Universe0                       0
weekday                         0
week                            0
month                           0
dtype: int64


In [129]:
def both_fillna(df):
    ffill_df = df.groupby('SecuritiesCode').fillna(method='ffill')
    ffill_df['SecuritiesCode'] = df['SecuritiesCode']
    both_fill_df = ffill_df.groupby('SecuritiesCode').fillna(method='bfill')
    both_fill_df['SecuritiesCode'] = df['SecuritiesCode']
    return both_fill_df

In [130]:
# df = df[~df['Target'].isnull()].reset_index(drop=True)
df['Target'].fillna(0, inplace=True)
df['ExpectedDividend'].fillna(0, inplace=True)
df = both_fillna(df)
df["high_rank"] = df.groupby("Date")["Target"].rank("dense", ascending=False).astype(int)

In [131]:
print(df.isnull().sum())

RowId                          0
Date                           0
Open                           0
High                           0
Low                            0
Close                          0
Volume                         0
AdjustmentFactor               0
ExpectedDividend               0
SupervisionFlag                0
Target                         0
Section/Products               0
NewMarketSegment               0
33SectorCode                   0
17SectorCode                   0
NewIndexSeriesSizeCode    303034
Universe0                      0
weekday                        0
week                           0
month                          0
SecuritiesCode                 0
high_rank                      0
dtype: int64


In [132]:
df

,RowId,Date,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,...,NewMarketSegment,33SectorCode,17SectorCode,NewIndexSeriesSizeCode,Universe0,weekday,week,month,SecuritiesCode,high_rank
0,20170104_1301,2017-01-04,2734.0,2755.0,2730.0,2742.0,31400,1.0,0.0,False,...,Prime Market,50,1,7,True,2,1,1,1301,858
1,20170104_1332,2017-01-04,568.0,576.0,563.0,571.0,2798500,1.0,0.0,False,...,Prime Market,50,1,4,True,2,1,1,1332,316
2,20170104_1333,2017-01-04,3150.0,3210.0,3140.0,3210.0,270800,1.0,0.0,False,...,Prime Market,50,1,4,True,2,1,1,1333,542
3,20170104_1376,2017-01-04,1510.0,1550.0,1510.0,1550.0,11300,1.0,0.0,False,...,Standard Market,50,1,7,True,2,1,1,1376,355
4,20170104_1377,2017-01-04,3270.0,3350.0,3270.0,3330.0,150800,1.0,0.0,False,...,Prime Market,50,1,6,True,2,1,1,1377,714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,514.0,528.0,513.0,528.0,44200,1.0,0.0,False,...,Prime Market,6100,14,7,True,4,48,12,9990,560
2332527,20211203_9991,2021-12-03,782.0,794.0,782.0,794.0,35900,1.0,0.0,False,...,Prime Market,6050,13,7,True,4,48,12,9991,1057
2332528,20211203_9993,2021-12-03,1690.0,1690.0,1645.0,1645.0,7200,1.0,0.0,False,...,Standard Market,6100,14,7,True,4,48,12,9993,1825
2332529,20211203_9994,2021-12-03,2388.0,2396.0,2380.0,2389.0,6500,1.0,0.0,False,...,Standard Market,6100,14,7,True,4,48,12,9994,1671


In [133]:
def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_price(df, price_cols=['Open', 'High', 'Low', 'Close']):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()

        for price_col in price_cols:
            # generate AdjustedClose
            df.loc[:, f"Adjusted{price_col}"] = (
                df["CumulativeAdjustmentFactor"] * df[price_col]
            ).map(lambda x: float(
                Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
            ))
            # reverse order
            df = df.sort_values("Date")
            # to fill AdjustedClose, replace 0 into np.nan
            df.loc[df[f"Adjusted{price_col}"] == 0, f"Adjusted{price_col}"] = np.nan
            # forward fill AdjustedClose
            df.loc[:, f"Adjusted{price_col}"] = df.loc[:, f"Adjusted{price_col}"].ffill()
        # return df["Date",  f"Adjusted{price_col}"]
        return df

    # generate Adjusted Prices
    price = price.sort_values(["Date", "SecuritiesCode"])
    AdjustedPrices = price.groupby("SecuritiesCode").apply(generate_adjusted_price).reset_index(drop=True)

    return AdjustedPrices

In [134]:
df = adjust_price(df)

In [135]:
# 騰落率
df["ror_1"] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(1)
df["ror_5"] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(5)
df["ror_10"] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(10)
df["ror_20"] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(20)
df["ror_40"] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(40)
df["ror_60"] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(60)
df["ror_100"] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(100)

In [157]:
# 売買代金
df["TradedAmount"] = df["AdjustedClose"] * df["Volume"]
df = df.replace([np.inf, -np.inf], np.nan)

groups = df.groupby('SecuritiesCode')
rolling_df = pd.DataFrame()
df["Amount_1"] = df["TradedAmount"]
rolling_df[['level_1', 'Amount_5']] = groups['TradedAmount'].rolling(5).mean().reset_index()[['level_1', 'TradedAmount']]
rolling_df['Amount_10'] = groups['TradedAmount'].rolling(10).mean().reset_index(drop=True)
rolling_df['Amount_20'] = groups["TradedAmount"].rolling(20).mean().reset_index(drop=True)
rolling_df['Amount_40'] = groups["TradedAmount"].rolling(40).mean().reset_index(drop=True)
rolling_df['Amount_60'] = groups["TradedAmount"].rolling(60).mean().reset_index(drop=True)
rolling_df = rolling_df.set_index('level_1').sort_values('level_1')
df = pd.concat([df, rolling_df], axis=1)
df["d_Amount"] = df["TradedAmount"]/df["Amount_20"]

In [165]:
# レンジ
df['PreviousClose'] = df.groupby('SecuritiesCode')['AdjustedClose'].shift(1)
df["range"] = (df[['PreviousClose', 'AdjustedHigh']].max(axis=1) - df[['PreviousClose', 'AdjustedLow']].min(axis=1)) / df['PreviousClose']
df = df.replace([np.inf, -np.inf], np.nan)

groups = df.groupby('SecuritiesCode')
rolling_df = pd.DataFrame()
df["atr_1"] = df["range"]
rolling_df[['level_1', "atr_5"]] = groups["range"].rolling(5).mean().reset_index()[['level_1', 'range']]
rolling_df["atr_10"] = groups["range"].rolling(10).mean().reset_index(drop=True)
rolling_df["atr_20"] = groups["range"].rolling(20).mean().reset_index(drop=True)
rolling_df["atr_40"] = groups["range"].rolling(40).mean().reset_index(drop=True)
rolling_df["atr_60"] = groups["range"].rolling(60).mean().reset_index(drop=True)
rolling_df["atr_100"] = groups["range"].rolling(100).mean().reset_index(drop=True)
df = pd.concat([df, rolling_df], axis=1)
df["d_atr"] = df["range"]/df["atr_20"]

In [ ]:
# ギャップレンジ
df["gap_range"] = (np.abs(df["AdjustedOpen"] - df["PreviousClose"])) / df["PreviousClose"]

groups = df.groupby('SecuritiesCode')
rolling_df = pd.DataFrame()
df["g_atr_1"] = df["gap_range"]
rolling_df[['level_1', "g_atr_5"]] = groups["range"].rolling(5).mean().reset_index()[['level_1', 'range']]
rolling_df["g_atr_10"] = groups["gap_range"].rolling(10).mean().reset_index(drop=True)
rolling_df["g_atr_20"] = groups["gap_range"].rolling(20).mean().reset_index(drop=True)
rolling_df["g_atr_40"] = groups["gap_range"].rolling(40).mean().reset_index(drop=True)
rolling_df["g_atr_60"] = groups["gap_range"].rolling(60).mean().reset_index(drop=True)
rolling_df["g_atr_100"] = groups["gap_range"].rolling(100).mean().reset_index(drop=True)
df = pd.concat([df, rolling_df], axis=1)

In [ ]:
# デイレンジ
df["day_range"] = (df["AdjustedHigh"] - df["AdjustedLow"]) / df["PreviousClose"]

groups = df.groupby('SecuritiesCode')
df["d_atr_1"] = df["day_range"]
df["d_atr_5"] = groups["day_range"].rolling(5).mean()
df["d_atr_10"] = groups["day_range"].rolling(10).mean()
df["d_atr_20"] = groups["day_range"].rolling(20).mean()
df["d_atr_40"] = groups["day_range"].rolling(40).mean()
df["d_atr_60"] = groups["day_range"].rolling(60).mean()
df["d_atr_100"] = groups["day_range"].rolling(100).mean()

In [ ]:
# ヒゲレンジ
df["hig_range"] = ((df["AdjustedHigh"] - df["AdjustedLow"]) - np.abs(df["AdjustedOpen"] - df["AdjustedClose"])) / df["PreviousClose"]

groups = df.groupby('SecuritiesCode')
df["h_atr_1"] = df["hig_range"]
df["h_atr_5"] = groups["hig_range"].rolling(5).mean()
df["h_atr_10"] = groups["hig_range"].rolling(10).mean()
df["h_atr_20"] = groups["hig_range"].rolling(20).mean()
df["h_atr_40"] = groups["hig_range"].rolling(40).mean()
df["h_atr_60"] = groups["hig_range"].rolling(60).mean()
df["h_atr_100"] = groups["hig_range"].rolling(100).mean()

In [ ]:
# ボラティリティ
groups = df.groupby('SecuritiesCode')
df["vola_5"] = groups["ror_1"].rolling(5).std()
df["vola_10"] = groups["ror_1"].rolling(10).std()
df["vola_20"] = groups["ror_1"].rolling(20).std()
df["vola_40"] = groups["ror_1"].rolling(40).std()
df["vola_60"] = groups["ror_1"].rolling(60).std()
df["vola_100"] = groups["ror_1"].rolling(100).std()

In [168]:
groups["ror_1"].rolling(5).std().reset_index()

,SecuritiesCode,level_1,ror_1
0,1301,0,NaN
1,1301,1865,NaN
2,1301,3730,NaN
3,1301,5595,NaN
4,1301,7460,NaN
...,...,...,...
2332526,9997,2324530,0.013831
2332527,9997,2326530,0.014534
2332528,9997,2328530,0.023026
2332529,9997,2330530,0.022590


In [169]:
# ボラティリティ
groups = df.groupby('SecuritiesCode')
rolling_df = pd.DataFrame()
rolling_df[['level_1', "vola_5"]] = groups["ror_1"].rolling(5).std().reset_index()[['level_1', 'ror_1']]
rolling_df["vola_10"] = groups["ror_1"].rolling(10).std().reset_index(drop=True)
rolling_df["vola_20"] = groups["ror_1"].rolling(20).std().reset_index(drop=True)
rolling_df["vola_40"] = groups["ror_1"].rolling(40).std().reset_index(drop=True)
rolling_df["vola_60"] = groups["ror_1"].rolling(60).std().reset_index(drop=True)
rolling_df["vola_100"] = groups["ror_1"].rolling(100).std().reset_index(drop=True)
df = pd.concat([df, rolling_df], axis=1)

In [ ]:
# HLバンド
groups = df.groupby('SecuritiesCode')
df["hl_5"] = groups["AdjustedHigh"].rolling(5).max() - groups["AdjustedLow"].rolling(5).min()
df["hl_10"] = groups["AdjustedHigh"].rolling(10).max() - groups["AdjustedLow"].rolling(10).min()
df["hl_20"] = groups["AdjustedHigh"].rolling(20).max() - groups["AdjustedLow"].rolling(20).min()
df["hl_40"] = groups["AdjustedHigh"].rolling(40).max() - groups["AdjustedLow"].rolling(40).min()
df["hl_60"] = groups["AdjustedHigh"].rolling(60).max() - groups["AdjustedLow"].rolling(60).min()
df["hl_100"] = groups["AdjustedHigh"].rolling(100).max() - groups["AdjustedLow"].rolling(100).min()

In [180]:
# HLバンド
groups = df.groupby('SecuritiesCode')
rolling_df = pd.DataFrame()
rolling_df[["level_1", "hl_5"]] = (groups["AdjustedHigh"].rolling(5).max() - groups["AdjustedLow"].rolling(5).min()).reset_index()[["level_1",  0]]
rolling_df["hl_10"] = (groups["AdjustedHigh"].rolling(10).max() - groups["AdjustedLow"].rolling(10).min()).reset_index(drop=True)
rolling_df["hl_20"] = (groups["AdjustedHigh"].rolling(20).max() - groups["AdjustedLow"].rolling(20).min()).reset_index(drop=True)
rolling_df["hl_40"] = (groups["AdjustedHigh"].rolling(40).max() - groups["AdjustedLow"].rolling(40).min()).reset_index(drop=True)
rolling_df["hl_60"] = (groups["AdjustedHigh"].rolling(60).max() - groups["AdjustedLow"].rolling(60).min()).reset_index(drop=True)
rolling_df["hl_100"] = (groups["AdjustedHigh"].rolling(100).max() - groups["AdjustedLow"].rolling(100).min()).reset_index(drop=True)
df = pd.concat([df, rolling_df], axis=1)

,level_1,0
0,0,NaN
1,1865,NaN
2,3730,NaN
3,5595,NaN
4,7460,35.0
...,...,...
2332526,2324530,54.0
2332527,2326530,54.0
2332528,2328530,51.0
2332529,2330530,42.0


In [ ]:
# マーケットインパクト
df["mi"] = df["range"] / (df["Volume"] * df["AdjustedClose"])
df = df.replace([np.inf, -np.inf], np.nan)

groups = df.groupby('SecuritiesCode')
df["mi_5"] = groups["mi"].rolling(5).mean()
df["mi_10"] = groups["mi"].rolling(10).mean()
df["mi_20"] = groups["mi"].rolling(20).mean()
df["mi_40"] = groups["mi"].rolling(40).mean()
df["mi_60"] = groups["mi"].rolling(60).mean()
df["mi_100"] = groups["mi"].rolling(100).mean()

In [ ]:
df[df['Close'] != df['AdjustedClose']]

,RowId,Date,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,...,hl_40,hl_60,hl_100,mi,mi_5,mi_10,mi_20,mi_40,mi_60,mi_100
7,20170104_1407,2017-01-04,857.0,877.0,851.0,866.0,147000,1.00,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20170104_1414,2017-01-04,4940.0,5060.0,4935.0,5050.0,119600,1.00,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,20170104_1435,2017-01-04,4165.0,4180.0,4100.0,4165.0,95700,1.00,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,20170104_1712,2017-01-04,1458.0,1490.0,1445.0,1483.0,23300,1.00,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,20170104_1716,2017-01-04,998.0,1024.0,998.0,1023.0,42600,1.00,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320370,20211125_9369,2021-11-25,2000.0,2009.0,1989.0,1990.0,69100,1.00,23.0,False,...,7186.0,8186.0,16844.0,1.451547e-10,9.309294e-10,9.906903e-10,6.891846e-10,5.083564e-10,6.648223e-10,6.098831e-10
2320697,20211126_2317,2021-11-26,2000.0,2027.0,1992.0,2016.0,178700,0.25,0.0,False,...,6837.0,6837.0,6837.0,1.944426e-10,3.433991e-10,2.612682e-10,2.509231e-10,4.373526e-10,3.569001e-10,6.396835e-10
2321169,20211126_4443,2021-11-26,12770.0,12800.0,12280.0,12560.0,138600,0.25,0.0,False,...,10464.0,10464.0,32064.0,9.356632e-11,2.078109e-10,2.367561e-10,3.027880e-10,3.834035e-10,3.632831e-10,3.117903e-10
2321999,20211126_7816,2021-11-26,8150.0,8390.0,8010.0,8350.0,833200,0.50,0.0,False,...,18249.0,18249.0,18249.0,1.343654e-11,3.414565e-10,7.149607e-10,4.629069e-10,4.237747e-10,6.416178e-10,5.632653e-10


In [ ]:
all_ta_features = (
    df[['SecuritiesCode', "AdjustedOpen", "AdjustedHigh", "AdjustedLow", "AdjustedClose", "Volume"]]
        .groupby('SecuritiesCode')
        .apply(
            lambda x: ta.add_all_ta_features(
                x, "AdjustedOpen", "AdjustedHigh", "AdjustedLow", "AdjustedClose", "Volume", fillna=False
            )
        )
)

/Users/rasuharu/Dev/kaggle/jpx_tokyo_market_prediction/.venv/lib/python3.9/site-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/rasuharu/Dev/kaggle/jpx_tokyo_market_prediction/.venv/lib/python3.9/site-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/Users/rasuharu/Dev/kaggle/jpx_tokyo_market_prediction/.venv/lib/python3.9/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/rasuharu/Dev/kaggle/jpx_tokyo_market_prediction/.venv/lib/python3.9/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this war

In [ ]:
all_ta_features.drop(columns=["AdjustedOpen", "AdjustedHigh", "AdjustedLow", "AdjustedClose", "Volume"], inplace=True)

In [ ]:
all_ta_features.columns

Index(['SecuritiesCode', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
       'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_vwap', 'volume_mfi',
       'volume_nvi', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
       'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_atr', 'volatility_ui',
       'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       'trend_ichimoku_base', 'trend_ichimoku_a', '

In [ ]:
df = pd.concat([df, all_ta_features.iloc[:, 1:]], axis=1)

In [ ]:
# df = reduce_mem_usage(df, True)

In [ ]:
df.memory_usage(index=True)

Index                     128
RowId                18660248
Date                 18660248
Open                 18660248
High                 18660248
                       ...   
momentum_pvo_hist    18660248
momentum_kama        18660248
others_dr            18660248
others_dlr           18660248
others_cr            18660248
Length: 183, dtype: int64

In [ ]:
# def both_fillna(df):
#     ffill_df = df.groupby('SecuritiesCode').fillna(method='ffill')
#     ffill_df['SecuritiesCode'] = df['SecuritiesCode']
#     both_fill_df = ffill_df.groupby('SecuritiesCode').fillna(method='bfill')
#     both_fill_df['SecuritiesCode'] = df['SecuritiesCode']
#     return both_fill_df

In [ ]:
def shift_period_pearson_corr(df, target_col, index, columns, period):
    # 2000銘柄が揃うのは2020-12-23以降
    df = df[df['Date'] >= '2020-12-23']
    pivot = df.pivot(index=index, columns=columns, values=target_col)
    pivot.fillna(method='ffill', inplace=True)
    pivot.fillna(method='bfill', inplace=True)

    target = pivot.iloc[period:, :].values
    # target_lag = pivot.shift(period).iloc[period:, :].values
    target_lag = pivot.iloc[:-period, :].values

    diff = target - target.mean()
    lag_diff = target_lag - target_lag.mean()

    lag_corr = np.dot(diff.T, lag_diff) / (np.sqrt(sum(diff ** 2)) * np.sqrt(sum(lag_diff ** 2)))
    for i in range(lag_corr.shape[0]):
        lag_corr[i, i] = 0.0

    return lag_corr

In [ ]:
def append_lag_corr_code(df, target_col='AdjustedClose', index='Date', columns='SecuritiesCode', period=1):
    new_df = df.copy()

    lag_corr = shift_period_pearson_corr(new_df, target_col, index, columns, period)

    lag_idxmax = lag_corr.argmax(axis=1)
    lag_idxmin = lag_corr.argmin(axis=1)

    lag_corr_master = pd.DataFrame()
    lag_corr_master['SecuritiesCode'] = sorted(new_df['SecuritiesCode'].unique())
    lag_corr_master = lag_corr_master.reset_index()

    lag_idxmax_dict = dict(zip(range(lag_corr_master.shape[0]), lag_idxmax))
    lag_idxmin_dict = dict(zip(range(lag_corr_master.shape[0]), lag_idxmin))
    code_master_dict = dict(zip(range(lag_corr_master.shape[0]),  sorted(new_df['SecuritiesCode'].unique())))
    lag_corr_master['lag_max_corr_code'] = lag_corr_master['index'].map(lag_idxmax_dict)
    lag_corr_master['lag_min_corr_code'] = lag_corr_master['index'].map(lag_idxmin_dict)
    lag_corr_master[f'{target_col}_lag{period}_max_corr_code'] = lag_corr_master['lag_max_corr_code'].map(code_master_dict)
    lag_corr_master[f'{target_col}_lag{period}_min_corr_code'] = lag_corr_master['lag_min_corr_code'].map(code_master_dict)
    lag_corr_master = lag_corr_master[['SecuritiesCode', f'{target_col}_lag{period}_max_corr_code', f'{target_col}_lag{period}_min_corr_code']]

    return new_df.merge(lag_corr_master, on="SecuritiesCode", how="left")


In [ ]:
def append_value_by_corr(df, target_col='', target_corr_cols=[], suffixes=('', '_corr'), is_rowid=False):
    new_df = df.copy()

    if is_rowid:
        target_dict = dict(zip(new_df['RowId'], new_df[target_col]))
        if isinstance(target_corr_cols, list):
            for col in target_corr_cols:
                new_df[f'{col}_RowId'] = new_df['Date'].str.replace(r'-', '') + '_' + new_df[col].astype(str)
                target_corr_col = col[:-5]
                new_df[target_corr_col] = new_df[f'{col}_RowId'].map(target_dict)
                new_df.drop(columns=[f'{col}_RowId'], inplace=True)
        return new_df
    else:
        dfs = []
        for corr_col in target_corr_cols:
            merge_df = new_df.merge(
                    new_df[['Date', 'SecuritiesCode', target_col]],
                    left_on=['Date', corr_col],
                    right_on=['Date', 'SecuritiesCode'],
                    suffixes=('', '_corr'),
                    how='left'
                ).iloc[:, -1].to_frame()
            merge_df.columns = [corr_col[:-10]]
            # new_df.drop(columns=[corr_col], inplace=True)
            dfs.append(
                merge_df
            )

        return pd.concat(dfs, axis=1)

In [ ]:
def unpivot_price(df):
    unpivot_df = pd.melt(
        df[['Date', 'SecuritiesCode', 'AdjustedOpen', 'AdjustedHigh', 'AdjustedLow', 'AdjustedClose']],
        id_vars=['Date', 'SecuritiesCode'],
        value_vars=['AdjustedOpen',	'AdjustedHigh', 'AdjustedLow', 'AdjustedClose']
    )
    return unpivot_df.sort_values(['Date', 'SecuritiesCode']).reset_index(drop=True)

In [ ]:
def calc_unpivot_target(df, method='original'):
    unpivot_df = unpivot_price(df)
    unpivot_df = unpivot_df.merge(
        df[['Date', 'SecuritiesCode', 'Target', 'high_rank']],
        on=['Date', 'SecuritiesCode'],
        how='left'
    )
    if method == 'original':
        return unpivot_df
    unpivot_group = unpivot_df.groupby(['SecuritiesCode', 'variable'])
    unpivot_df['Target'] = (unpivot_group['value'].diff() / unpivot_group.shift(1)['value'])
    unpivot_df['Target'].fillna(0, inplace=True)
    return unpivot_df

In [ ]:
def aggregate_window(df, group_col='SecuritiesCode', target_col='AdjustedClose', window=5):
    '''
    Aggregate target columns by window as feature engineering.
    '''
    agg_columns = ['mean', 'var', 'max', 'min', 'skew', 'kurt']
    agg_df = (
        df.groupby(group_col)[target_col]
            .rolling(window)
            .agg(agg_columns)
            .reset_index()
            .set_index('level_1')
            .sort_values('level_1')
    )
    agg_df.drop(columns=['SecuritiesCode'], inplace=True)
    agg_df.columns = [f'{target_col}_{col}_{window}' for col in agg_columns]

    ewm_agg_columns = ['mean', 'std']
    ewm_agg_df = (
        df.groupby(group_col)[target_col]
            .ewm(com=0.5)
            .agg(ewm_agg_columns)
            .reset_index()
            .set_index('level_1')
            .sort_values('level_1')
    )
    ewm_agg_df.drop(columns=['SecuritiesCode'], inplace=True)
    ewm_agg_df.columns = [f'{target_col}_ewm_{col}_{window}' for col in ewm_agg_columns]

    return pd.concat([df, agg_df, ewm_agg_df], axis=1)

In [ ]:
Open_lag1_corr = append_lag_corr_code(df, target_col='AdjustedOpen')
Close_lag1_corr = append_lag_corr_code(df, target_col='AdjustedClose')
Target_lag1_corr = append_lag_corr_code(df, target_col='Target')
high_rank_lag1_corr = append_lag_corr_code(df, target_col='high_rank')

In [ ]:
Open_corr = append_value_by_corr(Open_lag1_corr, target_col='AdjustedOpen', target_corr_cols=['AdjustedOpen_lag1_max_corr_code', 'AdjustedOpen_lag1_min_corr_code'])
Close_corr = append_value_by_corr(Close_lag1_corr, target_col='AdjustedClose', target_corr_cols=['AdjustedClose_lag1_max_corr_code', 'AdjustedClose_lag1_min_corr_code'])
Target_corr = append_value_by_corr(Target_lag1_corr, target_col='AdjustedClose', target_corr_cols=['Target_lag1_max_corr_code', 'Target_lag1_min_corr_code'])
high_rank_corr = append_value_by_corr(high_rank_lag1_corr, target_col='AdjustedClose', target_corr_cols=['high_rank_lag1_max_corr_code', 'high_rank_lag1_min_corr_code'])

In [ ]:
concat_df = pd.concat([df, Open_corr, Close_corr, Target_corr, high_rank_corr], axis=1)

In [ ]:
agg_df =  aggregate_window(concat_df)

In [ ]:
agg_df.isnull().sum()

RowId                          0
Date                           0
Open                           0
High                           0
Low                            0
                            ... 
AdjustedClose_min_5         8000
AdjustedClose_skew_5        9633
AdjustedClose_kurt_5        9633
AdjustedClose_ewm_mean_5       0
AdjustedClose_ewm_std_5     2000
Length: 198, dtype: int64

# ①集計後のデータを保存

In [ ]:
# val_date = val_df.iloc[0].Date

In [ ]:
# train_agg_df = agg_df[agg_df['Date'] < val_date]
# val_agg_df = agg_df[agg_df['Date'] >= val_date]

In [ ]:
agg_df.to_parquet('../Output/train_agg_df.parquet')
# val_agg_df.to_parquet('../Output/val_agg_df.parquet')

In [ ]:
df['SecuritiesCode'].sort_values().unique()[df.groupby(['SecuritiesCode'])['Volume'].mean() < 1000]

array([2761, 4365, 4781, 9441, 9537, 9733])

# 決算データの前処理・特徴量エンジニアリング

In [ ]:
def datediff(df, date='Date', disclosed_date='DisclosedDate'):
    df[[date, disclosed_date]] = df[[date, disclosed_date]].apply(pd.to_datetime) #if conversion required
    df[f'{disclosed_date}_diff'] = (df[date] - df[disclosed_date]).dt.days
    return df

In [ ]:
# fill_fin_df[['Date','DisclosedDate']] = fill_fin_df[['Date','DisclosedDate']].apply(pd.to_datetime) #if conversion required
# fill_fin_df['DisclosedDateDiff'] = (fill_fin_df['Date'] - fill_fin_df['DisclosedDate']).dt.days

In [ ]:
financials = pd.read_csv('../Input/train_files/financials.csv')
# val_financials = pd.read_csv('../Input/supplemental_files/financials.csv')

/var/folders/ym/2nhqmm5d6zn8mfrndqmv6l_40000gn/T/ipykernel_25442/2348900381.py:1: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  financials = pd.read_csv('../Input/train_files/financials.csv')


In [ ]:
# financials = pd.concat([train_financials, val_financials]).reset_index(drop=True)

In [ ]:
numeric_cols = [
    'NetSales', 'OperatingProfit',
    'OrdinaryProfit', 'Profit', 'EarningsPerShare', 'TotalAssets', 'Equity',
    'EquityToAssetRatio', 'BookValuePerShare',
    'ResultDividendPerShare1stQuarter',
    'ResultDividendPerShare2ndQuarter',
    'ResultDividendPerShare3rdQuarter',
    'ResultDividendPerShareFiscalYearEnd',
    'ResultDividendPerShareAnnual',
    'ForecastDividendPerShare1stQuarter',
    'ForecastDividendPerShare2ndQuarter',
    'ForecastDividendPerShare3rdQuarter',
    'ForecastDividendPerShareFiscalYearEnd',
    'ForecastDividendPerShareAnnual',
    'ForecastNetSales',
    'ForecastOperatingProfit',
    'ForecastOrdinaryProfit',
    'ForecastProfit',
    'ForecastEarningsPerShare',
    'NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock',
    'NumberOfTreasuryStockAtTheEndOfFiscalYear',
    'AverageNumberOfShares'
]
str_cols = [col for col in financials.columns if col not in numeric_cols]

In [ ]:
fin_numeric = pd.to_numeric(financials[numeric_cols].stack(), errors='coerce').unstack()
fin_str = financials[str_cols]
financials = pd.concat([fin_str, fin_numeric], axis=1)

In [ ]:
# financials['NetSales'] = financials['NetSales'].str.replace('-', '')
# financials[(financials['NetSales'].notnull()) & (financials['NetSales'].str.contains('-'))]['NetSales']

In [ ]:
financials_group = financials.groupby(['SecuritiesCode'])
financials['NetSalesRatio'] = (financials_group['NetSales'].diff() / financials_group.shift(1)['NetSales'])
# financials['NetSalesRatio'] = (financials_group['NetSales'] / financials_group.shift(1)['NetSales'])
# financials['Target'].fillna(0, inplace=True)

In [ ]:
financials['NetSalesRatio']

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
92951   -0.838979
92952    0.578755
92953         NaN
92954   -0.852007
92955    1.039328
Name: NetSalesRatio, Length: 92956, dtype: float64

In [ ]:
drop_cols = [col for col in agg_df.columns if col not in ['RowId', 'Date', 'SecuritiesCode']]

In [ ]:
fin_df = financials.drop(columns=['Date', 'SecuritiesCode'])

In [ ]:
agg_df.columns

Index(['RowId', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag',
       ...
       'high_rank_lag1_max', 'high_rank_lag1_min', 'AdjustedClose_mean_5',
       'AdjustedClose_var_5', 'AdjustedClose_max_5', 'AdjustedClose_min_5',
       'AdjustedClose_skew_5', 'AdjustedClose_kurt_5',
       'AdjustedClose_ewm_mean_5', 'AdjustedClose_ewm_std_5'],
      dtype='object', length=198)

In [ ]:
fin_agg_df = agg_df.merge(fin_df, left_on='RowId', right_on='DateCode', how='left')
fin_agg_df.drop_duplicates(subset='RowId', keep='last', inplace=True)

In [ ]:
fin_agg_df.shape

(2332531, 242)

In [ ]:
# fin_df = fin_agg_df.drop(columns=drop_cols)
# fill_fin_df = both_fillna(fin_df)

In [ ]:
fill_fin_df = both_fillna(fin_agg_df)

/var/folders/ym/2nhqmm5d6zn8mfrndqmv6l_40000gn/T/ipykernel_25442/3115717676.py:4: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  both_fill_df = ffill_df.groupby('SecuritiesCode').fillna(method='bfill')


In [ ]:
fill_fin_df = datediff(fill_fin_df)

In [ ]:
fill_fin_df['MarketCapitalization'] = fill_fin_df['AdjustedClose'] * fill_fin_df['NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock']

In [ ]:
fill_fin_df['MarketCapitalization/NetSales'] = fill_fin_df['MarketCapitalization'] / fill_fin_df['NetSales']

In [ ]:
fill_fin_df.shape

(2332531, 245)

# ②決算データの処理結果を保存

In [ ]:
# train_fin_df = fill_fin_df[fill_fin_df['Date'] < val_date]
# val_fin_df = fill_fin_df[fill_fin_df['Date'] >= val_date]

In [ ]:
fill_fin_df.to_parquet('../Output/financials_train_agg_df.parquet')
# val_fin_df.to_parquet('../Output/financials_val_agg_df.parquet')

# 縦持ちに変換

In [ ]:
unpivot_df = calc_unpivot_target(fill_fin_df, method='calc')

In [ ]:
fill_fin_df.shape

(2332531, 245)

In [ ]:
unpivot_df

,Date,SecuritiesCode,variable,value,Target,high_rank
0,2017-01-04,1301,AdjustedOpen,2734.0,0.000000,858
1,2017-01-04,1301,AdjustedHigh,2755.0,0.000000,858
2,2017-01-04,1301,AdjustedLow,2730.0,0.000000,858
3,2017-01-04,1301,AdjustedClose,2742.0,0.000000,858
4,2017-01-04,1332,AdjustedOpen,568.0,0.000000,316
...,...,...,...,...,...,...
9330119,2021-12-03,9994,AdjustedClose,2389.0,0.011431,1671
9330120,2021-12-03,9997,AdjustedOpen,690.0,0.013216,1394
9330121,2021-12-03,9997,AdjustedHigh,711.0,0.027457,1394
9330122,2021-12-03,9997,AdjustedLow,686.0,0.008824,1394


In [ ]:
unpivot_fin_df = pd.merge(
    unpivot_df,
    fill_fin_df,
    on=['Date', 'SecuritiesCode'],
    how='left',
    suffixes=('_unpivot', '')
)

# ③縦持ちに変換したデータを保存

In [ ]:
# train_unpivot_fin_df = unpivot_fin_df[unpivot_fin_df['Date'] < val_date]
# val_unpivot_fin_df = unpivot_fin_df[unpivot_fin_df['Date'] >= val_date]

In [ ]:
unpivot_fin_df.to_parquet('../Output/unpivot_train_df.parquet')
# val_unpivot_fin_df.to_parquet('../Output/unpivot_val_df.parquet')

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

In [ ]:
# df[df['SecuritiesCode'] == 1301]['Close'].plot()

In [ ]:
# df[df['SecuritiesCode'] == 4781]['Close'].plot()

In [ ]:
# import time

# from contextlib import contextmanager

# @contextmanager
# def timer(name: str):
#     t0 = time.time()
#     msg = f"[{name}] start"
#     print(msg)
#     yield

#     msg = f"[{name}] done in {time.time() - t0:.2f} s"
#     print(msg)


In [ ]:
# # MIC相関行列用に空のデータフレームを作成
# df_mic = pd.DataFrame(columns=pivot_target.columns, index=pivot_target.columns)

# # forループ内でMIC相関係数を求め，先ほどのデータフレームに格納
# for column in tqdm(pivot_target.columns):

#     for index in pivot_target.columns:

#         # MIC相関係数の計算
#         mine.compute_score(pivot_target[column], pivot_target[index])
#         # 計算結果をデータフレームに書き込み
#         df_mic.loc[index, column] = mine.mic()

In [ ]:
# df.groupby('SecuritiesCode')['high_rank'].mean().sort_values()

In [ ]:
# df.groupby('SecuritiesCode')['Target'].mean().sort_values()